In [1]:
import cv2
import time
from deepface import DeepFace
# optional: from fer import FER  # if you want to try FER-based emotion detector


In [2]:
def draw_label(img, text, x, y, bg_color=(0, 0, 0), text_color=(255,255,255), font_scale=0.6, thickness=1):
    font = cv2.FONT_HERSHEY_SIMPLEX
    (w, h), _ = cv2.getTextSize(text, font, font_scale, thickness)
    cv2.rectangle(img, (x, y - h - 10), (x + w + 6, y + 4), bg_color, -1)
    cv2.putText(img, text, (x + 2, y - 4), font, font_scale, text_color, thickness, cv2.LINE_AA)


In [3]:
# Try to use OpenCV's default face cascade. This file ships with opencv-python.
# If this fails, you can use DeepFace's detector_backend='opencv' and it'll use DNN or MTCNN.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_faces_opencv(gray_frame):
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(60,60))
    return faces


In [4]:
# This function uses DeepFace.analyze for a face ROI.
# Note: we purposely DO NOT request 'race' from DeepFace.
def analyze_face_with_deepface(face_img):
    # DeepFace expects BGR images (as OpenCV), but will handle conversions internally.
    # actions can include 'age', 'gender', 'emotion' (we exclude 'race').
    try:
        result = DeepFace.analyze(face_img, actions=['age','gender','emotion'], enforce_detection=False, detector_backend='opencv')
        # result is a dict; on single face DeepFace returns dict with keys 'age','gender','dominant_emotion', 'emotion' etc.
        return result
    except Exception as e:
        # In case DeepFace fails (e.g., model loads), return None
        print("DeepFace analyze error:", e)
        return None


In [5]:
%pip uninstall deepface -y
%pip install deepface==0.0.79


Found existing installation: deepface 0.0.92
Uninstalling deepface-0.0.92:
  Successfully uninstalled deepface-0.0.92
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
import time
from deepface import DeepFace

# ------------------------------
# Helper: draw text with background
# ------------------------------
def draw_label(frame, text, x, y, bg_color=(0, 0, 0), text_color=(255, 255, 255)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    thickness = 2
    (tw, th), baseline = cv2.getTextSize(text, font, scale, thickness)
    cv2.rectangle(frame, (x, y - th - baseline), (x + tw, y + baseline), bg_color, -1)
    cv2.putText(frame, text, (x, y), font, scale, text_color, thickness, cv2.LINE_AA)

# ------------------------------
# Face detector (OpenCV Haar)
# ------------------------------
def detect_faces_opencv(gray_frame):
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    return faces

# ------------------------------
# Real-time webcam loop
# ------------------------------
USE_DEEPFACE = True   # Toggle DeepFace analysis
SHOW_GENDER = True    # Toggle gender display

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("❌ Could not open webcam. Check device index or permissions.")

fps_time = time.time()
frame_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detect_faces_opencv(gray)

        for (x, y, w, h) in faces:
            pad = int(0.05 * w)
            x1, y1 = max(x - pad, 0), max(y - pad, 0)
            x2, y2 = min(x + w + pad, frame.shape[1]), min(y + h + pad, frame.shape[0])
            face_img = frame[y1:y2, x1:x2].copy()

            label_parts = []

            if USE_DEEPFACE:
                try:
                    res = DeepFace.analyze(
                        face_img,
                        actions=['age', 'gender', 'emotion'],  # add 'race' if needed
                        detector_backend='opencv',
                        enforce_detection=False
                    )
                    if isinstance(res, list):
                        res = res[0]  # take first face dict

                    # Age
                    age = res.get("age", None)
                    if age is not None:
                        label_parts.append(f"Age:{int(age)}")

                    # Gender
                    if SHOW_GENDER:
                        gender = res.get("dominant_gender", "")
                        if gender:
                            label_parts.append(gender.capitalize())

                    # Emotion
                    emotion = res.get("dominant_emotion", "")
                    if emotion:
                        label_parts.append(emotion.capitalize())

                except Exception as e:
                    print("DeepFace error:", e)

            # Draw bounding box + label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label_text = " | ".join(label_parts) if label_parts else "Face"
            draw_label(frame, label_text, x1, y1)

        # FPS counter
        frame_count += 1
        if frame_count % 10 == 0:
            now = time.time()
            fps = 10.0 / (now - fps_time)
            fps_time = now
            draw_label(frame, f"FPS: {fps:.1f}", 10, 30, bg_color=(50, 50, 50))

        cv2.imshow("Real-time Age/Gender/Emotion Demo", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 12.21it/s]
